In [168]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp

In [169]:
df = pd.read_csv('data.csv', index_col='Run')
df['molF [mmol/min]'] = df['VolF [mL/min]'] /0.08206/(273.15+25)
df.head(5)

,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2
Run,,,,,,,
0,0.3,300,30,1.226182,0.925,0.019,0.056
1,0.3,300,60,2.452364,0.948,0.013,0.039
2,0.3,300,90,3.678546,0.951,0.012,0.036
3,0.3,350,30,1.226182,0.795,0.051,0.154
4,0.3,350,60,2.452364,0.843,0.039,0.118


In [170]:
df['Conv'] = (1-df['xNH3'])/(1+df['xNH3'])
df.head(5)

,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv
Run,,,,,,,,
0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961
1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694
2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115
3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206
4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187


In [171]:
df['molF_NH3'] = df['molF [mmol/min]'] * (1-df['Conv'])
df['molF_N2'] = df['molF [mmol/min]'] * df['Conv'] * 0.5
df['molF_H2'] = df['molF [mmol/min]'] * df['Conv'] * 1.5

df.head(5)

,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,molF_N2,molF_H2
Run,,,,,,,,,,,
0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,0.023887,0.071660
1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,0.032732,0.098195
2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,0.046194,0.138582
3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,0.070019,0.210056
4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,0.104455,0.313365


In [172]:
df['xNH3_cal'] = df['molF_NH3'] / (df['molF_NH3'] + df['molF_N2'] + df['molF_H2'])
df['xN2_cal'] = df['molF_N2'] / (df['molF_NH3'] + df['molF_N2'] + df['molF_H2'])
df['xH2_cal'] = df['molF_H2'] / (df['molF_NH3'] + df['molF_N2'] + df['molF_H2'])

df.head(5)

,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,molF_N2,molF_H2,xNH3_cal,xN2_cal,xH2_cal
Run,,,,,,,,,,,,,,
0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,0.023887,0.071660,0.925,0.01875,0.05625
1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,0.032732,0.098195,0.948,0.01300,0.03900
2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,0.046194,0.138582,0.951,0.01225,0.03675
3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,0.070019,0.210056,0.795,0.05125,0.15375
4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,0.104455,0.313365,0.843,0.03925,0.11775


In [173]:
df['Fin [mmol/min]'] = df['molF [mmol/min]']
df['Fout [mmol/min]'] = df['molF_NH3'] + df['molF_N2'] + df['molF_H2']

df.head(5)

,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,molF_N2,molF_H2,xNH3_cal,xN2_cal,xH2_cal,Fin [mmol/min],Fout [mmol/min]
Run,,,,,,,,,,,,,,,,
0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,0.023887,0.071660,0.925,0.01875,0.05625,1.226182,1.273955
1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,0.032732,0.098195,0.948,0.01300,0.03900,2.452364,2.517827
2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,0.046194,0.138582,0.951,0.01225,0.03675,3.678546,3.770934
3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,0.070019,0.210056,0.795,0.05125,0.15375,1.226182,1.366219
4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,0.104455,0.313365,0.843,0.03925,0.11775,2.452364,2.661274


In [174]:
df['Vin [mL/min]'] = df['Fin [mmol/min]'] * 0.08206 * (273.15+df['Temp [degC]'])
df['Vout [mL/min]'] = df['Fout [mmol/min]'] * 0.08206 * (273.15+df['Temp [degC]'])


df.head(5)

,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,molF_N2,molF_H2,xNH3_cal,xN2_cal,xH2_cal,Fin [mmol/min],Fout [mmol/min],Vin [mL/min],Vout [mL/min]
Run,,,,,,,,,,,,,,,,,,
0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,0.023887,0.071660,0.925,0.01875,0.05625,1.226182,1.273955,57.670636,59.917543
1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,0.032732,0.098195,0.948,0.01300,0.03900,2.452364,2.517827,115.341271,118.420196
2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,0.046194,0.138582,0.951,0.01225,0.03675,3.678546,3.770934,173.011907,177.357157
3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,0.070019,0.210056,0.795,0.05125,0.15375,1.226182,1.366219,62.701660,69.862574
4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,0.104455,0.313365,0.843,0.03925,0.11775,2.452364,2.661274,125.403320,136.086078
